In [ ]:
# jefan 8/28/17
# purpose: extract training phase metadata and sketches produced by participants
# in fMRI training study (aka 'neurosketch')

In [1]:
from __future__ import division
import numpy as np
from numpy import *
import os
import PIL
from PIL import Image
import base64
import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches
from skimage import data, io, filters
import cStringIO
import pandas as pd
import pymongo as pm ## first establish ssh tunnel to Amazon EC2 instance

/Users/judithfan/anaconda/lib/python2.7/site-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
# load in experimental design pickle file
import cPickle
mdtd = cPickle.load(open('morph_drawing_training_design.pkl'))

In [3]:
# mongo globals
conn = pm.MongoClient(port=20809)
DBNAME = conn['during_morph_drawing_recognition']
COLNAME = DBNAME['fmri3.files']
coll=COLNAME
DATADIR = 'neurosketch_data_3'

In [15]:
# get worker list
patient_ids_1 = ['1121161_neurosketch', '1130161_neurosketch', 
'1201161_neurosketch', '1202161_neurosketch', '1203161_neurosketch',
'1206161_neurosketch', '1206162_neurosketch','1206163_neurosketch',
'1207161_neurosketch','1207162_neurosketch']
patient_ids_2 = ['1207162_neurosketch']
patient_ids_3 = ['0110171_neurosketch', '0110172_neurosketch',
'0111171_neurosketch','0112171_neurosketch', '0112172_neurosketch','0112173_neurosketch',
'0113171_neurosketch','0115172_neurosketch','0115174_neurosketch','0117171_neurosketch',
'0118171_neurosketch','0118172_neurosketch','0119171_neurosketch','0119172_neurosketch',
'0119173_neurosketch', '0119174_neurosketch','0120171_neurosketch','0120172_neurosketch',
'0120173_neurosketch','0123171_neurosketch','0123172_neurosketch','0123173_neurosketch',
'0124171_neurosketch','0125171_neurosketch','0125172_neurosketch']

patient_ids = patient_ids_1 + patient_ids_2 + patient_ids_3
workers = patient_ids

## manually add 2nd half of 0123172 to list ('0123172_neurosketch_2'), and merge later
workers = workers + ['0123172_neurosketch_2']

In [16]:
w = workers[0]
these = coll.find({'wID': w}).sort('trialNum')   
## manually add

'0123172_neurosketch_2'

In [22]:
_workers = workers[-2:]

In [23]:
for w in _workers:
    try:
        print 'Now analyzing ' + w + ' ...'
        # retrieve this participant's records from mongo
        these = coll.find({'wID': w}).sort('trialNum')   

        wID = []
        phase = []
        version = []
        category = []
        viewpoint = []
        trial = []
        trialDuration = []
        target = []
        competitor = []
        svgString = []
        pngString = []

        # loop through training trials and save relevant metadata
        for this in these:
            if this['phase']=='training':
                wID.append(this['wID'])
                phase.append(this['phase'])
                version.append(int(this['versionNum']))
                category.append(this['category'])
                viewpoint.append(int(this['viewpoint']))
                trial.append(int(this['trialNum']))
                trialDuration.append(float(this['trialDuration']))
                _target = gm.get_object_name(2,gm.get_object_index(this['morphline'],this['morphnum']))
                target.append(_target)
                trained_set = gm.getEndpoints(this['morphline'])
                _competitor = [i for i in trained_set if i != _target]
                competitor.append(_competitor[0])
                svgString.append(this['json'])
                pngString.append(this['imgData'])
        ## make pandas dataframe to store metadata
        X = pd.DataFrame([wID,phase,version,category,viewpoint,trial,trialDuration,
                         target,competitor,svgString,pngString])
        X = X.transpose()
        X.columns = ['wID','phase','version','category','viewpoint','trial','trialDuration',
                    'target','competitor','svgString','pngString']

        ## save out dataframe to csv
        pathdir = os.path.join('data',w)
        ## save out image as png
        if not os.path.exists(pathdir):
            os.makedirs(pathdir)
        fname = w + '_metadata.csv'
        filepath = os.path.join(pathdir,fname)
        X.to_csv(filepath)  
        ## loop through all sketches and save out as png's in subject specific folders
        for t in trial:
            imgData = X[X['trial']==t].pngString.values[0]
            im = Image.open(cStringIO.StringIO(base64.b64decode(imgData)))
            fig = plt.figure(figsize=(8,8))
            p = plt.subplot(1,1,1)
            plt.imshow(im)
            k = p.get_xaxis().set_ticklabels([])
            k = p.get_yaxis().set_ticklabels([])
            k = p.get_xaxis().set_ticks([])
            k = p.get_yaxis().set_ticks([])
            for spine in plt.gca().spines.values():
                spine.set_visible(False)
            pathdir = os.path.join('data',X[X['trial']==t].wID.values[0])
            ## save out image as png
            if not os.path.exists(pathdir):
                os.makedirs(pathdir)
            fname = X[X['trial']==t].wID.values[0]  + '_trial_' + \
            str(X[X['trial']==t].trial.values[0]) + '_' + X[X['trial']==t].target.values[0]
            filepath = os.path.join(pathdir,fname)
            print 'Saving to ' + filepath
            fig.savefig(filepath+'.png',bbox_inches='tight')  
            plt.close(fig)
    except:
        print('Something went wrong with subject ' + w)

Now analyzing 0125172_neurosketch ...
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_320_chair
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_321_bed
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_322_chair
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_323_bed
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_324_chair
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_325_bed
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_326_chair
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_327_bed
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_328_chair
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_329_bed
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_330_chair
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_331_bed
Saving to data/0125172_neurosketch/0125172_neurosketch_trial_332_chair
Saving to data/0125172_neurosketch/0125172_neuroske

In [89]:
## manually merge 0123172_neurosketch & 0123172_neurosketch_2 datasets because they had to be collected in two different javascript game sessions
import shutil

def manual_merge_0123172():
    try:
        shutil.move('data/0123172_neurosketch_2/', 'data/0123172_neurosketch/')
        files = os.listdir('data/0123172_neurosketch/0123172_neurosketch_2')
        ## move images
        for f in files:
            if f.endswith('.png'):
                _f = f.split('_')[0] + '_' + f.split('_')[1] + '_' + f.split('_')[3] + '_' + f.split('_')[4] + '_' + f.split('_')[5]
                shutil.copy(os.path.join('data/0123172_neurosketch/0123172_neurosketch_2',f),'data/0123172_neurosketch/' + _f)
        ## now splice together pandas dataframe        
        df1 = pd.read_csv('data/0123172_neurosketch/0123172_neurosketch_metadata.csv')
        df2 = pd.read_csv('data/0123172_neurosketch/0123172_neurosketch_2/0123172_neurosketch_2_metadata.csv')
        _df2 = df2[df2.trial>=340]
        df = pd.concat([df1,_df2],ignore_index=True)        

        # rename old truncated metadata csv
        os.rename('data/0123172_neurosketch/0123172_neurosketch_metadata.csv','data/0123172_neurosketch/0123172_neurosketch_metadata_old.csv')
        os.rename('data/0123172_neurosketch/0123172_neurosketch_metadata_old.csv','fragment_backup/0123172_neurosketch_metadata_old.csv')

        # write out new repaired csv
        df.to_csv('data/0123172_neurosketch/0123172_neurosketch_metadata.csv')

        # clean up, move fragments to fragment folder
        shutil.move('data/0123172_neurosketch/0123172_neurosketch_2/', 'fragment_backup/backup')
    except:
        print 'Maybe already did this... '
        pass